In [1]:
import os
import re
import pandas as pd
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [2]:
stop_words = ["i'm","it's","don't","she's","she'll","he'll","i'll","won't","he's",'ourselves', 'hers',
              'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 
              'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 
              'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off',
              'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 
              'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 
              'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 
              'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 
              'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on',
              'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 
              'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too',
              'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if',
              'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here',
              'than','que'] 

In [4]:
expletive_set = ['fuck','shit','bitch']

In [8]:
# by_file_length_data - Gets number of words in each file
# by_year_occur_data - Gets occurences of each word in each decade
# by_file_occur_data - Gets occurences of each word in each file
# by_file_sentiment_data - Gets sentiment data in each file
# by_file_expletives_data - Gets expletive counts in each file
# by_file_occur_bigram_data - Gets occurences of bigrams in each file

by_file_length_flag = True
by_year_occur_flag = True
by_file_occur_flag = True
by_file_occur_bigram_flag = False
by_file_sentiment_flag = True
by_file_expletives_flag = False

remove_stop_words_flag = True

if not by_file_occur_flag:
    by_file_expletives_flag = False


path = 'song_lyrics_new/'
lyric_files = os.listdir(path)
by_file_length_data = pd.DataFrame(columns=['word_count'])
by_year_occur_data = pd.DataFrame(index=['total occurences'])
by_file_occur_data = pd.DataFrame(index=['total occurences'])
by_file_occur_bigram_data = pd.DataFrame(index=['total occurences'])
by_file_sentiment_data = pd.DataFrame(columns = ['polarity', 'subjectivity'])


files_length = len(lyric_files)
complete_files = 0

for lyric_file in lyric_files:
    if not complete_files % 10:
        print(f'Completed {complete_files} of {files_length} files.')
        
    with open(path + lyric_file) as lyrics:
        # Getting by song row name.
        year = re.sub(r'[0-9]$', '0', lyric_file.replace('.txt','')[-4:])
        if re.search(r'_0$', year):
            year = 'Null'
        by_year_row = f"{year}'s occurences"
        
        # Getting by file row name.
        by_file_row = f"{lyric_file.replace('.txt', '')}"
        
        # Formatting lyrics.
        words = lyrics.read()
        words_array = re.findall(r"[A-z][A-z'][A-z']+", re.sub(r'[[\]]', '', words.lower()))

        
        if by_file_length_flag:
            # By file - number of words
            if not by_file_row in by_file_length_data.index:
                new_index = []
                for current in by_file_length_data.index:
                    new_index.append(current)
                new_index.append(by_file_row)
                by_file_length_data = by_file_length_data.reindex(index=new_index, fill_value=0)

            num_of_words = len(words_array)
            by_file_length_data['word_count'][by_file_row] = num_of_words

        
        # By file - sentiment analysis
        if by_file_sentiment_flag:
            if not by_file_row in by_file_sentiment_data.index:
                new_index = []
                for current in by_file_sentiment_data.index:
                    new_index.append(current)
                new_index.append(by_file_row)
                by_file_sentiment_data = by_file_sentiment_data.reindex(index=new_index, fill_value=0)

            opinion = TextBlob(words) # analyzer=NaiveBayesAnalyzer()
            by_file_sentiment_data['polarity'][by_file_row] = opinion.sentiment[0]
            by_file_sentiment_data['subjectivity'][by_file_row] = opinion.sentiment[1]

        
        # Going through lyrics.
        for position, word in enumerate(words_array):
            # By year - occurences
            if by_year_occur_flag:
                if not word in by_year_occur_data.columns:
                    by_year_occur_data[word] = 0

                by_year_occur_data[word]['total occurences'] += 1
                if not by_year_row in by_year_occur_data.index:
                    new_index = []
                    for current in by_year_occur_data.index:
                        new_index.append(current)
                    new_index.append(by_year_row)
                    by_year_occur_data = by_year_occur_data.reindex(index=new_index, fill_value=0)
                by_year_occur_data[word][by_year_row] += 1


            # By file - occurences
            if by_file_occur_flag:
                if not word in by_file_occur_data.columns:
                    by_file_occur_data[word] = 0

                by_file_occur_data[word]['total occurences'] += 1
                if not by_file_row in by_file_occur_data.index:
                    new_index = []
                    for current in by_file_occur_data.index:
                        new_index.append(current)
                    new_index.append(by_file_row)
                    by_file_occur_data = by_file_occur_data.reindex(index=new_index, fill_value=0)
                by_file_occur_data[word][by_file_row] += 1
                
            # By file - bigram occurences
            if by_file_occur_bigram_flag:
                try:
                    bigram = word + ' ' + words_array[position + 1]
                except:
                    break

                if not bigram in by_file_occur_bigram_data.columns:
                    by_file_occur_bigram_data[bigram] = 0

                by_file_occur_bigram_data[bigram]['total occurences'] += 1
                if not by_file_row in by_file_occur_bigram_data.index:
                    new_index = []
                    for current in by_file_occur_bigram_data.index:
                        new_index.append(current)
                    new_index.append(by_file_row)
                    by_file_occur_bigram_data = by_file_occur_bigram_data.reindex(index=new_index, fill_value=0)
                by_file_occur_bigram_data[bigram][by_file_row] += 1
                
                

    complete_files += 1

if by_file_expletives_flag:
    by_file_expletives_data = by_file_occur_data[expletive_set]

if remove_stop_words_flag:
    if by_file_occur_flag:
        by_file_occur_data = by_file_occur_data.drop(stop_words, axis=1, errors='ignore')

    if by_year_occur_flag:
        by_year_occur_data = by_year_occur_data.drop(stop_words, axis=1, errors='ignore')
        


print(f'Completed {complete_files} of {files_length} files.')

Completed 0 of 4157 files.
Completed 10 of 4157 files.
Completed 20 of 4157 files.
Completed 30 of 4157 files.
Completed 40 of 4157 files.
Completed 50 of 4157 files.
Completed 60 of 4157 files.
Completed 70 of 4157 files.
Completed 80 of 4157 files.
Completed 90 of 4157 files.
Completed 100 of 4157 files.
Completed 110 of 4157 files.
Completed 120 of 4157 files.
Completed 130 of 4157 files.
Completed 140 of 4157 files.
Completed 150 of 4157 files.
Completed 160 of 4157 files.
Completed 170 of 4157 files.
Completed 180 of 4157 files.
Completed 190 of 4157 files.
Completed 200 of 4157 files.
Completed 210 of 4157 files.
Completed 220 of 4157 files.
Completed 230 of 4157 files.
Completed 240 of 4157 files.
Completed 250 of 4157 files.
Completed 260 of 4157 files.
Completed 270 of 4157 files.
Completed 280 of 4157 files.
Completed 290 of 4157 files.
Completed 300 of 4157 files.
Completed 310 of 4157 files.
Completed 320 of 4157 files.
Completed 330 of 4157 files.
Completed 340 of 4157 fil

Completed 2770 of 4157 files.
Completed 2780 of 4157 files.
Completed 2790 of 4157 files.
Completed 2800 of 4157 files.
Completed 2810 of 4157 files.
Completed 2820 of 4157 files.
Completed 2830 of 4157 files.
Completed 2840 of 4157 files.
Completed 2850 of 4157 files.
Completed 2860 of 4157 files.
Completed 2870 of 4157 files.
Completed 2880 of 4157 files.
Completed 2890 of 4157 files.
Completed 2900 of 4157 files.
Completed 2910 of 4157 files.
Completed 2920 of 4157 files.
Completed 2930 of 4157 files.
Completed 2940 of 4157 files.
Completed 2950 of 4157 files.
Completed 2960 of 4157 files.
Completed 2970 of 4157 files.
Completed 2980 of 4157 files.
Completed 2990 of 4157 files.
Completed 3000 of 4157 files.
Completed 3010 of 4157 files.
Completed 3020 of 4157 files.
Completed 3030 of 4157 files.
Completed 3040 of 4157 files.
Completed 3050 of 4157 files.
Completed 3060 of 4157 files.
Completed 3070 of 4157 files.
Completed 3080 of 4157 files.
Completed 3090 of 4157 files.
Completed 

In [131]:
import numpy as np
tf_idf = pd.DataFrame()
path = 'song_lyrics_new/'
lyric_files = os.listdir(path)
files_length = len(lyric_files)
complete_files = 0

for lyric_file in lyric_files:
    print("hi")
    if not complete_files % 10:
        print(f'Completed {complete_files} of {files_length} files.')

        # Getting by file row name.
        by_file_row = f"{lyric_file.replace('.txt', '')}"
        with open(path + lyric_file) as lyrics:
        # Formatting lyrics.
            words = lyrics.read()
            words_array = re.findall(r"[A-z][A-z'][A-z']+", re.sub(r'[[\]]', '', words.lower()))


            for word in words_array:
                if not word in tf_idf.columns:
                    tf_idf[word] = 0
                if not by_file_row in tf_idf.index:
                    new_index = []
                    for current in tf_idf.index:
                        new_index.append(current)
                    new_index.append(by_file_row)
                    tf_idf = tf_idf.reindex(index=new_index, fill_value=0)
                if word in by_file_occur_data:
                    tf_idf[word][by_file_row] = by_file_occur_data[word][by_file_row] * np.log(files_length/by_file_occur_data[word].astype(bool).sum(axis=0))
                else:
                    tf_idf[word][by_file_row] = 0
    complete_files += 1

        

hi
Completed 0 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 10 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 20 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 30 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 40 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 50 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 60 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 70 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 80 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 90 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 100 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 110 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 120 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 130 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 140 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 150 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 160 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Compl

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1390 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1400 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1410 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1420 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1430 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1440 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1450 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1460 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1470 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1480 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1490 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1500 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1510 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1520 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1530 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 1540 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2760 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2770 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2780 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2790 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2800 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2810 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2820 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2830 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2840 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2850 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2860 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2870 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2880 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2890 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2900 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 2910 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 4140 of 4157 files.
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Completed 4150 of 4157 files.
hi
hi
hi
hi
hi
hi


In [122]:
import numpy as np
tf_idf_year = pd.DataFrame()
path = 'song_lyrics_new/'
lyric_files = os.listdir(path)
files_length = len(lyric_files)
complete_files = 0

for lyric_file in lyric_files:
    if not complete_files % 10:
        print(f'Completed {complete_files} of {files_length} files.')
        # Getting year.
    year = re.sub(r'[0-9]$', '0', lyric_file.replace('.txt','')[-4:])
    print(year)

    if re.search(r'_0$', year):
        year = 'Null'
    print(year)
    by_year_row = f"{year}'s occurences"
    with open(path + lyric_file) as lyrics:
    # Formatting lyrics.
        words = lyrics.read()
        words_array = re.findall(r"[A-z][A-z'][A-z']+", re.sub(r'[[\]]', '', words.lower()))


        for word in words_array:
            if not word in tf_idf_year.columns:
                tf_idf_year[word] = 0

                if not by_year_row in tf_idf_year.index:
                    new_index = []
                    for current in tf_idf_year.index:
                        new_index.append(current)
                    new_index.append(by_year_row)
                    tf_idf_year = tf_idf_year.reindex(index=new_index, fill_value=0)
                if word in by_year_occur_data:
                    tf_idf_year[word][by_year_row] = by_year_occur_data[word][by_year_row] * np.log(files_length/by_year_occur_data[word].astype(bool).sum(axis=0))
                else:
                    tf_idf_year[word][by_year_row] = 0
    complete_files += 1

Completed 0 of 4157 files.
1990
1990
2000
2000
2000
2000
2010
2010
2010
2010
1990
1990
2000
2000
1970
1970
1950
1950
1960
1960
Completed 10 of 4157 files.
1990
1990
1980
1980
1980
1980
2000
2000
1980
1980
2010
2010
1990
1990
1990
1990
2000
2000
2000
2000
Completed 20 of 4157 files.
2000
2000
2000
2000
1990
1990
2000
2000
2000
2000
2000
2000
1990
1990
1960
1960
me_0
Null
2000
2000
Completed 30 of 4157 files.
1990
1990
2000
2000
2000
2000
1990
1990
2000
2000
2000
2000
2000
2000
1980
1980
1970
1970
2000
2000
Completed 40 of 4157 files.
1990
1990
2000
2000
2000
2000
2000
2000
2000
2000
1990
1990
2010
2010
1990
1990
2000
2000
2000
2000
Completed 50 of 4157 files.
2000
2000
2000
2000
1980
1980
1960
1960
1980
1980
1980
1980
2000
2000
2000
2000
2010
2010
2000
2000
Completed 60 of 4157 files.
2000
2000
2000
2000
2000
2000
2000
2000
1990
1990
2000
2000
1980
1980
1980
1980
2000
2000
1990
1990
Completed 70 of 4157 files.
2000
2000
1990
1990
2000
2000
1980
1980
1960
1960
2000
2000
2000
2000
2000
20

1960
1960
2000
2000
1990
1990
Completed 640 of 4157 files.
1980
1980
1980
1980
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
1950
1950
2010
2010
1990
1990
Completed 650 of 4157 files.
1990
1990
1980
1980
1990
1990
1990
1990
2000
2000
2000
2000
1970
1970
2000
2000
2000
2000
1990
1990
Completed 660 of 4157 files.
an_0
Null
1990
1990
2000
2000
2000
2000
1990
1990
1990
1990
2000
2000
1980
1980
2000
2000
2000
2000
Completed 670 of 4157 files.
1980
1980
1950
1950
1990
1990
1990
1990
1980
1980
2000
2000
2000
2000
2000
2000
1990
1990
1990
1990
Completed 680 of 4157 files.
2000
2000
1970
1970
1980
1980
2000
2000
1960
1960
1980
1980
1960
1960
1960
1960
1960
1960
2000
2000
Completed 690 of 4157 files.
2000
2000
2000
2000
1990
1990
2000
2000
1970
1970
1980
1980
1990
1990
2000
2000
2000
2000
2000
2000
Completed 700 of 4157 files.
1970
1970
1990
1990
2000
2000
2000
2000
1960
1960
2000
2000
2000
2000
1990
1990
1990
1990
2000
2000
Completed 710 of 4157 files.
2000
2000
2010
2010
2000
2000
1980
198

1920
1920
2000
2000
1970
1970
1940
1940
Completed 1280 of 4157 files.
1990
1990
2000
2000
1990
1990
2000
2000
1980
1980
2000
2000
2000
2000
1920
1920
1990
1990
2000
2000
Completed 1290 of 4157 files.
1980
1980
2000
2000
2000
2000
1990
1990
2000
2000
1990
1990
2000
2000
1990
1990
2000
2000
2000
2000
Completed 1300 of 4157 files.
2000
2000
2000
2000
2000
2000
1990
1990
1990
1990
2000
2000
2000
2000
1990
1990
2000
2000
2000
2000
Completed 1310 of 4157 files.
1980
1980
1990
1990
1980
1980
1990
1990
1990
1990
2000
2000
2000
2000
1990
1990
1980
1980
1990
1990
Completed 1320 of 4157 files.
2000
2000
2000
2000
1990
1990
1990
1990
2000
2000
1970
1970
2000
2000
2000
2000
1990
1990
2000
2000
Completed 1330 of 4157 files.
1980
1980
1980
1980
2000
2000
2000
2000
2000
2000
1990
1990
1980
1980
2010
2010
1990
1990
2000
2000
Completed 1340 of 4157 files.
2000
2000
2000
2000
1990
1990
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2010
2010
1980
1980
Completed 1350 of 4157 files.
an_0
Null
2000
2000


1990
1990
Completed 1910 of 4157 files.
2000
2000
2000
2000
1990
1990
2000
2000
2000
2000
1990
1990
2000
2000
1990
1990
2000
2000
1990
1990
Completed 1920 of 4157 files.
2010
2010
1990
1990
1990
1990
1990
1990
1980
1980
2000
2000
1970
1970
2000
2000
1990
1990
2000
2000
Completed 1930 of 4157 files.
1960
1960
1990
1990
1990
1990
1990
1990
1990
1990
2000
2000
1990
1990
1970
1970
2000
2000
1990
1990
Completed 1940 of 4157 files.
2000
2000
2010
2010
2000
2000
1960
1960
1980
1980
1990
1990
2000
2000
1980
1980
2000
2000
1980
1980
Completed 1950 of 4157 files.
2000
2000
1990
1990
1990
1990
2000
2000
2000
2000
1970
1970
2000
2000
2000
2000
1990
1990
2000
2000
Completed 1960 of 4157 files.
1990
1990
2000
2000
1990
1990
1990
1990
1990
1990
2000
2000
1990
1990
1960
1960
1990
1990
2000
2000
Completed 1970 of 4157 files.
2000
2000
2010
2010
2000
2000
1990
1990
2010
2010
1990
1990
1990
1990
2000
2000
2000
2000
1990
1990
Completed 1980 of 4157 files.
2000
2000
1970
1970
1990
1990
1990
1990
1990
1990


2000
2000
2000
2000
1980
1980
2000
2000
2000
2000
1990
1990
2000
2000
Completed 2550 of 4157 files.
1990
1990
1970
1970
2000
2000
2000
2000
2000
2000
2000
2000
1990
1990
1980
1980
2000
2000
2000
2000
Completed 2560 of 4157 files.
2000
2000
2000
2000
1990
1990
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
1990
1990
1990
1990
Completed 2570 of 4157 files.
1960
1960
1990
1990
1970
1970
2000
2000
1980
1980
1980
1980
1980
1980
2000
2000
1990
1990
1990
1990
Completed 2580 of 4157 files.
2000
2000
1990
1990
2000
2000
1970
1970
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
Completed 2590 of 4157 files.
2000
2000
2000
2000
1970
1970
1980
1980
1960
1960
1990
1990
1970
1970
1990
1990
1990
1990
2000
2000
Completed 2600 of 4157 files.
2000
2000
2000
2000
1980
1980
1980
1980
2000
2000
2000
2000
1990
1990
2000
2000
1980
1980
1990
1990
Completed 2610 of 4157 files.
1990
1990
2010
2010
1970
1970
2000
2000
2000
2000
2000
2000
2000
2000
wn_0
Null
1990
1990
1980
1980
Completed 2620 of 41

2000
2000
1980
1980
1990
1990
1990
1990
2000
2000
Completed 3180 of 4157 files.
2010
2010
2000
2000
2000
2000
2000
2000
1980
1980
2000
2000
1990
1990
2000
2000
2000
2000
2000
2000
Completed 3190 of 4157 files.
2000
2000
1980
1980
1980
1980
2000
2000
1990
1990
2000
2000
2000
2000
2010
2010
1990
1990
2000
2000
Completed 3200 of 4157 files.
1980
1980
2000
2000
2000
2000
1970
1970
1960
1960
1970
1970
ns_0
Null
1990
1990
1980
1980
1990
1990
Completed 3210 of 4157 files.
2000
2000
1990
1990
2000
2000
2000
2000
1980
1980
eo_0
Null
1980
1980
1980
1980
2000
2000
1980
1980
Completed 3220 of 4157 files.
1960
1960
2000
2000
1990
1990
1970
1970
2010
2010
1990
1990
1980
1980
1990
1990
1960
1960
1950
1950
Completed 3230 of 4157 files.
1970
1970
2000
2000
1990
1990
2000
2000
2000
2000
2000
2000
1990
1990
2000
2000
2000
2000
2000
2000
Completed 3240 of 4157 files.
2000
2000
1990
1990
2000
2000
2010
2010
2000
2000
1990
1990
1990
1990
2000
2000
2000
2000
1980
1980
Completed 3250 of 4157 files.
2000
2000


2000
2000
2010
2010
1990
1990
Completed 3810 of 4157 files.
1980
1980
2000
2000
2000
2000
1990
1990
2010
2010
2000
2000
1990
1990
1990
1990
2000
2000
1990
1990
Completed 3820 of 4157 files.
1990
1990
2010
2010
2000
2000
1980
1980
2000
2000
2000
2000
1970
1970
1990
1990
1990
1990
2000
2000
Completed 3830 of 4157 files.
2000
2000
2000
2000
2000
2000
1990
1990
et_0
Null
1970
1970
1990
1990
2000
2000
2000
2000
1980
1980
Completed 3840 of 4157 files.
1990
1990
2000
2000
2000
2000
2000
2000
1990
1990
2000
2000
1990
1990
1990
1990
1950
1950
1990
1990
Completed 3850 of 4157 files.
2010
2010
1990
1990
2000
2000
1990
1990
1990
1990
2000
2000
1960
1960
1970
1970
1990
1990
2000
2000
Completed 3860 of 4157 files.
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
2000
1980
1980
2000
2000
2000
2000
Completed 3870 of 4157 files.
2000
2000
1970
1970
1990
1990
2010
2010
2000
2000
2000
2000
1990
1990
1990
1990
2010
2010
2010
2010
Completed 3880 of 4157 files.
1990
1990
2000
2000
1990
1990


In [121]:

by_file_occur_data = by_file_occur_data.drop(stop_words, axis=1, errors='ignore')

by_year_occur_data = by_year_occur_data.drop(stop_words, axis=1, errors='ignore')
tf_idf = tf_idf.drop(stop_words, axis=1, errors='ignore')
tf_idf_year = tf_idf_year.drop(stop_words, axis=1, errors='ignore')



In [134]:
# by_year_occur_data.sort_values(by = ["1990's occurences"],axis = 1)
# # by_year_occur_data
# by_file_sentiment_data.sort_values(by = ['polarity'],axis=0, ascending = False)
tf_idf.sort_values(by = ["'57 Chevrolet_Billie Jo Spears_1990"], axis = 1, ascending = False)
# tf_idf_year
# tf_idf

,chevrolet,owe,old,still,tasted,wish,today,picture,we'd,cars,...,forth,exorcists,name's,slant,showed,welcome,mat,shrinking,deal,confusi
'57 Chevrolet_Billie Jo Spears_1990,48,26,19,12,12,9,9,8,8,8,...,0,0,0,0,0,0,0,0,0,0
(The Grave Prelude)_Mobb Deep_1995,0,0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11th Street_Open Hand_2000,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2 Fists Full Of Nothing_Bruisers_1993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32 Lines (Album Version)_Sophie B Hawkins_1999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6TH Avenue Nightmare_Pennywise_2005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Dustland Fairytale_The Killers_2008,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Martyr For My Love For You_The White Stripes_2007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Veces Vuelvo_Catupecu Machu_2004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
About Last Night (LP Version)_Vitamin C_1999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# by_year_occur_data.sort_values(by = ["1920's occurences"],axis = 1, ascending = False)


In [97]:
import pandas as pd
import csv
# by_file_sentiment_data = pd.read_csv("by_file_sentiment_data.csv", quoting=csv.QUOTE_NONE, error_bad_lines=False)
by_file_length_data = pd.read_csv("by_file_length_data.csv", index_col = 0)
# by_file_occur_data = pd.read_csv("by_file_occur_data.csv", index_col = 0)


In [138]:
# by_year_occur_data.set_index('Unnamed: 0').T
# by_file_occur_data
tf_idf.to_csv("tf_idf.csv")


In [64]:
# path = 'song_lyrics_new/'
# lyric_files = os.listdir(path)
# files_length = len(lyric_files)
# complete_files = 0

# for lyric_file in lyric_files:
#     if not complete_files % 10:
#         print(f'Completed {complete_files} of {files_length} files.')
#         # Getting year.
#         year = re.sub(r'[0-9]$', '0', lyric_file.replace('.txt','')[-4:])
#         print(year)